In [1]:
conda init powershell

no change     C:\Users\Paulo\miniconda3\Scripts\conda.exe
no change     C:\Users\Paulo\miniconda3\Scripts\conda-env.exe
no change     C:\Users\Paulo\miniconda3\Scripts\conda-script.py
no change     C:\Users\Paulo\miniconda3\Scripts\conda-env-script.py
no change     C:\Users\Paulo\miniconda3\condabin\conda.bat
no change     C:\Users\Paulo\miniconda3\Library\bin\conda.bat
no change     C:\Users\Paulo\miniconda3\condabin\_conda_activate.bat
no change     C:\Users\Paulo\miniconda3\condabin\rename_tmp.bat
no change     C:\Users\Paulo\miniconda3\condabin\conda_auto_activate.bat
no change     C:\Users\Paulo\miniconda3\condabin\conda_hook.bat
no change     C:\Users\Paulo\miniconda3\Scripts\activate.bat
no change     C:\Users\Paulo\miniconda3\condabin\activate.bat
no change     C:\Users\Paulo\miniconda3\condabin\deactivate.bat
no change     C:\Users\Paulo\miniconda3\Scripts\activate
no change     C:\Users\Paulo\miniconda3\Scripts\deactivate
no change     C:\Users\Paulo\miniconda3\etc\profile.d\

In [ ]:
conda create --name myenv python=3.12 

In [ ]:
!conda activate myenv
!pip install ir_datasets datasets sentence-transformers scikit-learn pandas tqdm ir_datasets faiss-cpu

In [4]:
!pip install datasets sentence-transformers scikit-learn pandas tqdm ir_datasets faiss-cpu

  Using cached ir_datasets-0.5.11-py3-none-any.whl.metadata (12 kB)
  Using cached faiss_cpu-1.11.0.post1-cp313-cp313-win_amd64.whl.metadata (5.1 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached inscriptis-2.6.0-py3-none-any.whl.metadata (25 kB)
  Using cached lxml-6.0.0-cp313-cp313-win_amd64.whl.metadata (6.8 kB)
  Using cached trec_car_tools-2.6-py3-none-any.whl.metadata (640 bytes)
  Using cached warc3_wet-0.2.5-py3-none-any.whl.metadata (2.2 kB)
  Using cached warc3_wet_clueweb09-0.2.5-py3-none-any.whl
  Using cached zlib_state-0.1.9.tar.gz (9.5 kB)
  

  error: subprocess-exited-with-error
  
  × Building wheel for zlib-state (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [32 lines of output]
      C:\Users\Paulo\AppData\Local\Temp\pip-build-env-4gulcf17\overlay\Lib\site-packages\setuptools\dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license expression:
      
              License :: OSI Approved :: MIT License
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ********************************************************************************
      
      !!
        self._finalize_license_expression()
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-3

In [ ]:
import numpy as np
import faiss
import json
import pandas as pd
import os
import pickle
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import logging
from typing import Dict, List, Tuple, Any, Optional
import torch
import re
import time
from math import log
from collections import defaultdict
import ir_datasets
import gc
from datasets import load_dataset
import warnings
warnings.filterwarnings("ignore")

# Set up comprehensive logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('msmarco_evaluation.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class BM25:
    """Optimized BM25 implementation with proper vectorization"""

    def __init__(self, k1: float = 1.5, b: float = 0.75):
        self.k1 = k1
        self.b = b
        self.corpus_size = 0
        self.avg_doc_len = 0
        self.doc_lens = []
        self.idf = {}
        self.doc_term_freqs = []

    def fit(self, corpus: List[List[str]]):
        """Fit BM25 on tokenized corpus"""
        logger.info(f"Fitting BM25 model on {len(corpus)} documents...")
        self.corpus_size = len(corpus)

        # Calculate document lengths
        self.doc_lens = [len(doc) for doc in corpus]
        self.avg_doc_len = sum(self.doc_lens) / len(self.doc_lens) if self.doc_lens else 0

        logger.info(f"Average document length: {self.avg_doc_len:.2f} tokens")

        # Calculate term frequencies for each document
        self.doc_term_freqs = []
        df = defaultdict(int)  # document frequency

        logger.info("Computing term frequencies...")
        for i, doc in enumerate(tqdm(corpus, desc="Processing documents for BM25")):
            term_freq = defaultdict(int)
            for term in doc:
                term_freq[term] += 1
            self.doc_term_freqs.append(dict(term_freq))

            # Count document frequency
            for term in set(doc):
                df[term] += 1

            # Memory cleanup every 100k documents
            if i % 100000 == 0 and i > 0:
                logger.info(f"Processed {i} documents for BM25...")
                gc.collect()

        # Calculate IDF
        logger.info(f"Computing IDF scores for {len(df)} unique terms...")
        for term, freq in df.items():
            self.idf[term] = log(self.corpus_size / freq)

        logger.info(f"BM25 model fitted successfully with {len(self.idf)} terms")

    def get_scores(self, query: List[str]) -> np.ndarray:
        """Get BM25 scores for query against all documents"""
        scores = np.zeros(self.corpus_size)

        for i in range(self.corpus_size):
            score = 0
            doc_len = self.doc_lens[i]

            for term in query:
                if term in self.doc_term_freqs[i] and term in self.idf:
                    tf = self.doc_term_freqs[i][term]
                    idf = self.idf[term]

                    # BM25 formula
                    numerator = tf * (self.k1 + 1)
                    denominator = tf + self.k1 * (1 - self.b + self.b * doc_len / self.avg_doc_len)
                    score += idf * (numerator / denominator)

            scores[i] = score

        return scores

class MSMarcoEvaluator:
    def __init__(self, model_name: str = "all-MiniLM-L6-v2", save_dir: str = "./msmarco_full_eval"):
        """
        Initialize evaluator for FULL MS MARCO dataset using robust data loading
        """
        self.model_name = model_name
        self.model: Optional[SentenceTransformer] = None
        self.dense_index: Optional[faiss.Index] = None
        self.bm25: Optional[BM25] = None

        # Document storage (train set - passages)
        self.documents: List[str] = []
        self.doc_ids: List[str] = []
        self.doc_id_to_index: Dict[str, int] = {}
        self.tokenized_docs: List[List[str]] = []
        self.embeddings: Optional[np.ndarray] = None

        # Query and relevance storage (dev set)
        self.dev_queries: Dict[str, str] = {}
        self.qrels: Dict[str, Dict[str, int]] = {}

        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)

        logger.info(f"Initializing MS MARCO Evaluator with save directory: {save_dir}")
        self._load_model()

    def _load_model(self):
        """Load the sentence transformer model."""
        try:
            logger.info(f"Loading sentence transformer model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            logger.info(f"Model loaded successfully. Max sequence length: {self.model.max_seq_length}")

            # Log device information
            device = 'cuda' if torch.cuda.is_available() else 'cpu'
            if device == 'cuda':
                logger.info(f"CUDA available - GPU: {torch.cuda.get_device_name()}")
                logger.info(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
            else:
                logger.info("Using CPU for inference")

        except Exception as e:
            logger.error(f"Error loading model: {e}")
            raise

    def _preprocess_text(self, text: str) -> List[str]:
        """Enhanced text preprocessing for BM25."""
        if not text:
            return []

        # Remove special characters and convert to lowercase
        text = re.sub(r'[^\w\s]', ' ', text.lower())
        # Split and filter tokens
        tokens = [token for token in text.split() if len(token) >= 2 and len(token) <= 20]
        return tokens

    def _safe_text_processing(self, text: str) -> str:
        """Safely process text to handle encoding issues."""
        try:
            if isinstance(text, bytes):
                text = text.decode('utf-8', errors='ignore')
            elif not isinstance(text, str):
                text = str(text)

            # Clean the text
            text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # Remove non-ASCII
            text = ' '.join(text.split())  # Normalize whitespace
            return text.strip()
        except Exception as e:
            logger.warning(f"Error processing text: {e}")
            return ""

    def load_full_msmarco_data(self):
        """Load the FULL MS MARCO dataset using HuggingFace datasets for reliability"""
        logger.info("Loading FULL MS MARCO dataset...")

        # Check if we have cached data
        if self.load_cached_data():
            logger.info("Successfully loaded cached MS MARCO data")
            return True

        try:
            logger.info("="*80)
            logger.info("LOADING MS MARCO PASSAGE RANKING DATASET - FULL VERSION")
            logger.info("This will download and process the complete dataset")
            logger.info("="*80)

            # Method 1: Load using HuggingFace datasets (most reliable)
            logger.info("Loading MS MARCO collection using HuggingFace datasets...")
            
            # Load the collection (passages/documents)
            logger.info("Step 1: Loading passage collection...")
            try:
                collection_dataset = load_dataset("microsoft/msmarco", "collection", split="collection")
                logger.info(f"Loaded collection with {len(collection_dataset)} passages")
                
                # Process collection in batches
                batch_size = 50000
                processed_count = 0
                
                for i in tqdm(range(0, len(collection_dataset), batch_size), desc="Processing collection"):
                    batch = collection_dataset[i:i+batch_size]
                    
                    for j, (doc_id, text) in enumerate(zip(batch['id'], batch['text'])):
                        clean_text = self._safe_text_processing(text)
                        if clean_text and len(clean_text) > 10:
                            self.documents.append(clean_text)
                            self.doc_ids.append(str(doc_id))
                            self.doc_id_to_index[str(doc_id)] = len(self.documents) - 1
                            processed_count += 1
                    
                    # Log progress and save intermediate data
                    if processed_count % 100000 == 0:
                        logger.info(f"Processed {processed_count} documents...")
                        self._save_intermediate_data()
                        gc.collect()
                
                logger.info(f"Collection loaded: {len(self.documents)} documents")
                
            except Exception as e:
                logger.error(f"Error loading collection via HuggingFace: {e}")
                # Fallback to ir_datasets
                logger.info("Falling back to ir_datasets...")
                return self._load_with_ir_datasets()

            # Step 2: Load queries
            logger.info("Step 2: Loading dev queries...")
            try:
                queries_dataset = load_dataset("microsoft/msmarco", "queries.dev", split="dev")
                logger.info(f"Loaded {len(queries_dataset)} dev queries")
                
                for item in tqdm(queries_dataset, desc="Processing queries"):
                    query_id = str(item['id'])
                    query_text = self._safe_text_processing(item['text'])
                    if query_text:
                        self.dev_queries[query_id] = query_text
                        
                logger.info(f"Dev queries loaded: {len(self.dev_queries)}")
                
            except Exception as e:
                logger.error(f"Error loading queries: {e}")
                # Try alternative approach
                try:
                    queries_dataset = load_dataset("microsoft/msmarco-dev", "queries", split="dev")
                    for item in tqdm(queries_dataset, desc="Processing queries (alt method)"):
                        query_id = str(item['query_id'])
                        query_text = self._safe_text_processing(item['query'])
                        if query_text:
                            self.dev_queries[query_id] = query_text
                    logger.info(f"Dev queries loaded (alt): {len(self.dev_queries)}")
                except Exception as e2:
                    logger.warning(f"Could not load queries: {e2}")

            # Step 3: Load qrels
            logger.info("Step 3: Loading relevance judgments (qrels)...")
            try:
                qrels_dataset = load_dataset("microsoft/msmarco", "qrels.dev", split="dev")
                logger.info(f"Loaded {len(qrels_dataset)} qrels")
                
                for item in tqdm(qrels_dataset, desc="Processing qrels"):
                    query_id = str(item['query_id'])
                    doc_id = str(item['corpus_id'])
                    relevance = int(item['score'])
                    
                    if query_id not in self.qrels:
                        self.qrels[query_id] = {}
                    self.qrels[query_id][doc_id] = relevance
                    
                logger.info(f"Qrels loaded: {len(self.qrels)} queries with relevance judgments")
                
            except Exception as e:
                logger.error(f"Error loading qrels: {e}")
                # Try alternative
                try:
                    qrels_dataset = load_dataset("microsoft/msmarco-dev", "qrels", split="dev")
                    for item in tqdm(qrels_dataset, desc="Processing qrels (alt)"):
                        query_id = str(item['query_id'])
                        doc_id = str(item['corpus_id'])
                        relevance = int(item['score'])
                        
                        if query_id not in self.qrels:
                            self.qrels[query_id] = {}
                        self.qrels[query_id][doc_id] = relevance
                    logger.info(f"Qrels loaded (alt): {len(self.qrels)}")
                except Exception as e2:
                    logger.warning(f"Could not load qrels: {e2}")

            # Step 4: Tokenize documents for BM25
            logger.info("Step 4: Tokenizing documents for BM25...")
            self.tokenized_docs = []
            batch_size = 10000

            for i in tqdm(range(0, len(self.documents), batch_size), desc="Tokenizing documents"):
                batch = self.documents[i:i+batch_size]
                tokenized_batch = [self._preprocess_text(doc) for doc in batch]
                self.tokenized_docs.extend(tokenized_batch)

                # Memory cleanup
                if i % 100000 == 0 and i > 0:
                    logger.info(f"Tokenized {i} documents...")
                    gc.collect()

            # Save all data
            self._save_all_data()

            logger.info("="*80)
            logger.info("MS MARCO DATASET LOADING COMPLETED!")
            logger.info(f"Documents: {len(self.documents):,}")
            logger.info(f"Queries: {len(self.dev_queries):,}")
            logger.info(f"QRels: {len(self.qrels):,}")
            logger.info("="*80)
            
            return True

        except Exception as e:
            logger.error(f"Error loading MS MARCO dataset: {e}")
            return False

    def _load_with_ir_datasets(self):
        """Fallback method using ir_datasets with proper error handling"""
        try:
            logger.info("Loading with ir_datasets as fallback...")
            
            # Try different dataset variations
            dataset_names = [
                "msmarco-passage",
                "msmarco-passage/dev",
                "msmarco-passage/train"
            ]
            
            dataset = None
            for dataset_name in dataset_names:
                try:
                    logger.info(f"Trying to load dataset: {dataset_name}")
                    dataset = ir_datasets.load(dataset_name)
                    logger.info(f"Successfully loaded: {dataset_name}")
                    break
                except Exception as e:
                    logger.warning(f"Failed to load {dataset_name}: {e}")
                    continue
            
            if dataset is None:
                logger.error("Could not load any MS MARCO dataset variant")
                return False
            
            # Load documents
            logger.info("Loading documents...")
            doc_count = 0
            try:
                if hasattr(dataset, 'docs_iter'):
                    for doc in tqdm(dataset.docs_iter(), desc="Loading documents"):
                        try:
                            clean_text = self._safe_text_processing(doc.text)
                            if clean_text and len(clean_text) > 10:
                                self.documents.append(clean_text)
                                self.doc_ids.append(doc.doc_id)
                                self.doc_id_to_index[doc.doc_id] = len(self.documents) - 1
                                doc_count += 1
                                
                                if doc_count % 100000 == 0:
                                    logger.info(f"Loaded {doc_count} documents...")
                                    self._save_intermediate_data()
                                    gc.collect()
                                    
                        except Exception as e:
                            logger.debug(f"Error processing document: {e}")
                            continue
                else:
                    logger.warning("Dataset does not have docs_iter method")
            except Exception as e:
                logger.error(f"Error loading documents: {e}")
            
            logger.info(f"Loaded {len(self.documents)} documents")
            
            # Load queries with multiple approaches
            logger.info("Loading queries...")
            queries_loaded = False
            
            # Method 1: Try queries_iter
            try:
                if hasattr(dataset, 'queries_iter'):
                    for query in tqdm(dataset.queries_iter(), desc="Loading queries"):
                        try:
                            clean_query = self._safe_text_processing(query.text)
                            if clean_query:
                                self.dev_queries[query.query_id] = clean_query
                        except Exception as e:
                            logger.debug(f"Error processing query: {e}")
                            continue
                    queries_loaded = True
                    logger.info(f"Loaded {len(self.dev_queries)} queries via queries_iter")
                else:
                    logger.warning("Dataset does not have queries_iter method")
            except Exception as e:
                logger.warning(f"queries_iter failed: {e}")
            
            # Method 2: Try loading dev queries separately
            if not queries_loaded:
                try:
                    logger.info("Trying to load dev queries separately...")
                    dev_dataset = ir_datasets.load("msmarco-passage/dev")
                    if hasattr(dev_dataset, 'queries_iter'):
                        for query in tqdm(dev_dataset.queries_iter(), desc="Loading dev queries"):
                            try:
                                clean_query = self._safe_text_processing(query.text)
                                if clean_query:
                                    self.dev_queries[query.query_id] = clean_query
                            except Exception as e:
                                continue
                        queries_loaded = True
                        logger.info(f"Loaded {len(self.dev_queries)} dev queries")
                except Exception as e:
                    logger.warning(f"Dev queries loading failed: {e}")
            
            # Method 3: Try accessing queries as a dataset property
            if not queries_loaded:
                try:
                    logger.info("Trying to access queries as dataset property...")
                    if hasattr(dataset, 'queries'):
                        queries_data = dataset.queries
                        for query_id, query_text in queries_data.items():
                            clean_query = self._safe_text_processing(query_text)
                            if clean_query:
                                self.dev_queries[query_id] = clean_query
                        queries_loaded = True
                        logger.info(f"Loaded {len(self.dev_queries)} queries via dataset property")
                except Exception as e:
                    logger.warning(f"Dataset property access failed: {e}")
            
            if not queries_loaded:
                logger.warning("Could not load queries via ir_datasets. Will continue with documents only.")
            
            # Load qrels with multiple approaches
            logger.info("Loading qrels...")
            qrels_loaded = False
            
            # Method 1: Try qrels_iter
            try:
                if hasattr(dataset, 'qrels_iter'):
                    for qrel in tqdm(dataset.qrels_iter(), desc="Loading qrels"):
                        try:
                            if qrel.query_id not in self.qrels:
                                self.qrels[qrel.query_id] = {}
                            self.qrels[qrel.query_id][qrel.doc_id] = qrel.relevance
                        except Exception as e:
                            logger.debug(f"Error processing qrel: {e}")
                            continue
                    qrels_loaded = True
                    logger.info(f"Loaded qrels for {len(self.qrels)} queries")
                else:
                    logger.warning("Dataset does not have qrels_iter method")
            except Exception as e:
                logger.warning(f"qrels_iter failed: {e}")
            
            # Method 2: Try loading dev qrels separately
            if not qrels_loaded:
                try:
                    logger.info("Trying to load dev qrels separately...")
                    dev_dataset = ir_datasets.load("msmarco-passage/dev")
                    if hasattr(dev_dataset, 'qrels_iter'):
                        for qrel in tqdm(dev_dataset.qrels_iter(), desc="Loading dev qrels"):
                            try:
                                if qrel.query_id not in self.qrels:
                                    self.qrels[qrel.query_id] = {}
                                self.qrels[qrel.query_id][qrel.doc_id] = qrel.relevance
                            except Exception as e:
                                continue
                        qrels_loaded = True
                        logger.info(f"Loaded dev qrels for {len(self.qrels)} queries")
                except Exception as e:
                    logger.warning(f"Dev qrels loading failed: {e}")
            
            if not qrels_loaded:
                logger.warning("Could not load qrels via ir_datasets.")
            
            # If we have documents but no queries/qrels, try to load them via HuggingFace as a final attempt
            if len(self.documents) > 0 and (len(self.dev_queries) == 0 or len(self.qrels) == 0):
                logger.info("Attempting to load queries/qrels via HuggingFace as final fallback...")
                try:
                    # Load queries
                    if len(self.dev_queries) == 0:
                        queries_dataset = load_dataset("microsoft/msmarco", "queries.dev", split="dev")
                        for item in tqdm(queries_dataset, desc="Loading HF queries"):
                            query_id = str(item['id'])
                            query_text = self._safe_text_processing(item['text'])
                            if query_text:
                                self.dev_queries[query_id] = query_text
                        logger.info(f"Loaded {len(self.dev_queries)} queries via HuggingFace")
                    
                    # Load qrels
                    if len(self.qrels) == 0:
                        qrels_dataset = load_dataset("microsoft/msmarco", "qrels.dev", split="dev")
                        for item in tqdm(qrels_dataset, desc="Loading HF qrels"):
                            query_id = str(item['query_id'])
                            doc_id = str(item['corpus_id'])
                            relevance = int(item['score'])
                            
                            if query_id not in self.qrels:
                                self.qrels[query_id] = {}
                            self.qrels[query_id][doc_id] = relevance
                        logger.info(f"Loaded qrels for {len(self.qrels)} queries via HuggingFace")
                        
                except Exception as e:
                    logger.error(f"HuggingFace fallback also failed: {e}")
            
            success = len(self.documents) > 0
            if success:
                logger.info(f"ir_datasets loading completed with:")
                logger.info(f"- Documents: {len(self.documents)}")
                logger.info(f"- Queries: {len(self.dev_queries)}")
                logger.info(f"- QRels: {len(self.qrels)}")
            
            return success
            
        except Exception as e:
            logger.error(f"ir_datasets fallback failed completely: {e}")
            return False
    def _save_intermediate_data(self):
        """Save intermediate data during loading"""
        try:
            temp_path = os.path.join(self.save_dir, 'temp_documents.pkl')
            with open(temp_path, 'wb') as f:
                pickle.dump({
                    'documents': self.documents,
                    'doc_ids': self.doc_ids,
                    'doc_id_to_index': self.doc_id_to_index
                }, f)
        except Exception as e:
            logger.warning(f"Error saving intermediate data: {e}")

    def _save_all_data(self):
        """Save all loaded data to cache"""
        try:
            # Save documents
            docs_path = os.path.join(self.save_dir, 'full_documents.pkl')
            logger.info(f"Saving {len(self.documents)} documents to cache...")
            with open(docs_path, 'wb') as f:
                pickle.dump({
                    'documents': self.documents,
                    'doc_ids': self.doc_ids,
                    'doc_id_to_index': self.doc_id_to_index,
                    'tokenized_docs': self.tokenized_docs
                }, f)

            # Save queries and qrels
            queries_path = os.path.join(self.save_dir, 'dev_queries.pkl')
            with open(queries_path, 'wb') as f:
                pickle.dump(self.dev_queries, f)

            qrels_path = os.path.join(self.save_dir, 'qrels.pkl')
            with open(qrels_path, 'wb') as f:
                pickle.dump(self.qrels, f)

            logger.info("All data saved to cache successfully")

        except Exception as e:
            logger.error(f"Error saving data to cache: {e}")

    def load_cached_data(self) -> bool:
        """Load cached data if available"""
        docs_path = os.path.join(self.save_dir, 'full_documents.pkl')
        queries_path = os.path.join(self.save_dir, 'dev_queries.pkl')
        qrels_path = os.path.join(self.save_dir, 'qrels.pkl')

        if all(os.path.exists(p) for p in [docs_path, queries_path, qrels_path]):
            try:
                logger.info("Loading cached data...")

                # Load documents
                with open(docs_path, 'rb') as f:
                    doc_data = pickle.load(f)
                    self.documents = doc_data['documents']
                    self.doc_ids = doc_data['doc_ids']
                    self.doc_id_to_index = doc_data['doc_id_to_index']
                    self.tokenized_docs = doc_data.get('tokenized_docs', [])

                # Load queries
                with open(queries_path, 'rb') as f:
                    self.dev_queries = pickle.load(f)

                # Load qrels
                with open(qrels_path, 'rb') as f:
                    self.qrels = pickle.load(f)

                logger.info(f"Cached data loaded: {len(self.documents)} docs, {len(self.dev_queries)} queries, {len(self.qrels)} qrel sets")

                # Tokenize documents if not cached
                if not self.tokenized_docs:
                    logger.info("Tokenizing documents for BM25...")
                    self.tokenized_docs = []
                    batch_size = 10000
                    for i in tqdm(range(0, len(self.documents), batch_size), desc="Tokenizing"):
                        batch = self.documents[i:i+batch_size]
                        tokenized_batch = [self._preprocess_text(doc) for doc in batch]
                        self.tokenized_docs.extend(tokenized_batch)
                        
                        if i % 100000 == 0 and i > 0:
                            gc.collect()
                    
                    # Save updated cache
                    with open(docs_path, 'wb') as f:
                        pickle.dump({
                            'documents': self.documents,
                            'doc_ids': self.doc_ids,
                            'doc_id_to_index': self.doc_id_to_index,
                            'tokenized_docs': self.tokenized_docs
                        }, f)

                return True

            except Exception as e:
                logger.error(f"Error loading cached data: {e}")
                return False

        return False

    def build_dense_index(self, batch_size: int = 64):
        """Build optimized dense FAISS index with IVF for large collection"""
        logger.info("Building dense FAISS index...")

        if not self.documents:
            raise ValueError("No documents loaded")

        embeddings_path = os.path.join(self.save_dir, 'embeddings.npy')
        index_path = os.path.join(self.save_dir, 'faiss_index.bin')

        # Try to load existing embeddings and index
        if os.path.exists(embeddings_path) and os.path.exists(index_path):
            try:
                logger.info("Loading existing embeddings and index...")
                self.embeddings = np.load(embeddings_path)
                self.dense_index = faiss.read_index(index_path)

                if len(self.embeddings) == len(self.documents):
                    logger.info(f"Loaded existing index with {self.dense_index.ntotal} documents")
                    return
                else:
                    logger.warning("Embedding count mismatch. Rebuilding...")
            except Exception as e:
                logger.error(f"Error loading cached index: {e}. Rebuilding...")

        # Generate embeddings with memory management
        logger.info("Generating embeddings...")
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        logger.info(f"Using device: {device}")

        # Process in chunks to handle memory
        chunk_size = 25000 if device == 'cuda' else 5000
        all_embeddings = []

        logger.info(f"Processing {len(self.documents)} documents in chunks of {chunk_size}")

        for i in tqdm(range(0, len(self.documents), chunk_size), desc="Generating embeddings"):
            chunk_docs = self.documents[i:i+chunk_size]
            try:
                chunk_embeddings = self.model.encode(
                    chunk_docs,
                    convert_to_numpy=True,
                    show_progress_bar=False,
                    batch_size=batch_size,
                    device=device,
                    normalize_embeddings=True
                )
                all_embeddings.append(chunk_embeddings)
                
                # Log progress
                if i % 100000 == 0:
                    logger.info(f"Generated embeddings for {i + len(chunk_docs)} documents")
                
            except Exception as e:
                logger.error(f"Error generating embeddings for chunk {i}: {e}")
                continue

            # Memory cleanup
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            gc.collect()

        # Concatenate all embeddings
        logger.info("Concatenating embeddings...")
        self.embeddings = np.vstack(all_embeddings)

        # Save embeddings
        logger.info("Saving embeddings...")
        np.save(embeddings_path, self.embeddings)

        # Build IVF index for large collection
        dimension = self.embeddings.shape[1]
        num_docs = len(self.documents)

        logger.info(f"Building IVF index for {num_docs} documents with dimension {dimension}")

        # Calculate optimal number of centroids
        n_list = min(max(int(4 * np.sqrt(num_docs)), 4000), num_docs // 50)
        logger.info(f"Using {n_list} centroids for IVF index")

        # Create IVF index
        quantizer = faiss.IndexFlatIP(dimension)
        self.dense_index = faiss.IndexIVFFlat(quantizer, dimension, n_list, faiss.METRIC_INNER_PRODUCT)

        # Train the index
        logger.info(f"Training IVF index...")
        training_data = self.embeddings[::max(1, len(self.embeddings) // 1000000)]  # Sample for training
        self.dense_index.train(training_data)

        # Add vectors to index in batches
        logger.info("Adding vectors to index...")
        batch_size_add = 100000
        for i in tqdm(range(0, len(self.embeddings), batch_size_add), desc="Adding to index"):
            end_idx = min(i + batch_size_add, len(self.embeddings))
            self.dense_index.add(self.embeddings[i:end_idx])
            
            if i % 500000 == 0 and i > 0:
                logger.info(f"Added {i} vectors to index")

        # Save index
        faiss.write_index(self.dense_index, index_path)
        logger.info(f"IVF index built and saved with {self.dense_index.ntotal} documents")

    def build_bm25_index(self):
        """Build BM25 index"""
        logger.info("Building BM25 index...")

        bm25_path = os.path.join(self.save_dir, 'bm25_model.pkl')

        # Try to load existing BM25 model
        if os.path.exists(bm25_path):
            try:
                logger.info("Loading existing BM25 model...")
                with open(bm25_path, 'rb') as f:
                    self.bm25 = pickle.load(f)
                logger.info("Loaded existing BM25 model")
                return
            except Exception as e:
                logger.error(f"Error loading cached BM25 model: {e}. Rebuilding...")

        if not self.tokenized_docs:
            raise ValueError("Documents must be loaded and tokenized before building BM25 index.")

        # Build BM25 model
        self.bm25 = BM25()
        self.bm25.fit(self.tokenized_docs)

        # Save BM25 model
        try:
            logger.info("Saving BM25 model...")
            with open(bm25_path, 'wb') as f:
                pickle.dump(self.bm25, f)
            logger.info("BM25 index built and cached successfully")
        except Exception as e:
            logger.error(f"Error saving BM25 model: {e}")

    def search_dense(self, query: str, k: int = 1000) -> List[Tuple[str, float, str]]:
        """Dense retrieval using semantic similarity with IVF search"""
        if self.dense_index is None:
            raise ValueError("Dense index not built")

        # Set nprobe for IVF index (search more centroids for better recall)
        if hasattr(self.dense_index, 'nprobe'):
            self.dense_index.nprobe = min(100, self.dense_index.nlist // 2)

        # Generate query embedding
        query_embedding = self.model.encode(
            [query],
            convert_to_numpy=True,
            normalize_embeddings=True
        )

        # Search
        scores, indices = self.dense_index.search(query_embedding, k)

        results = []
        for score, idx in zip(scores[0], indices[0]):
            if 0 <= idx < len(self.doc_ids):
                results.append((
                    self.doc_ids[idx],
                    float(score),
                    self.documents[idx]
                ))

        return results

    def search_bm25(self, query: str, k: int = 1000) -> List[Tuple[str, float, str]]:
        """Sparse retrieval using BM25"""
        if self.bm25 is None:
            raise ValueError("BM25 index not built. Call build_bm25_index() first.")

        tokenized_query = self._preprocess_text(query)
        if not tokenized_query:
            return []

        scores = self.bm25.get_scores(tokenized_query)

        # Get top k results
        top_indices = np.argsort(scores)[::-1][:k]

        results = []
        for idx in top_indices:
            if scores[idx] > 0:
                results.append((
                    self.doc_ids[idx],
                    float(scores[idx]),
                    self.documents[idx]
                ))

        return results

    def search_hybrid(self, query: str, k: int = 100, alpha: float = 0.5, candidate_k: int = 500) -> List[Tuple[str, float, str]]:
        """
        Hybrid retrieval: Get 500 candidates from dense, then re-rank with BM25
        """
        if self.dense_index is None or self.bm25 is None:
            raise ValueError("Both dense and BM25 indices must be built for hybrid search.")

        # Step 1: Get candidates using dense retrieval
        logger.debug(f"Getting {candidate_k} candidates from dense search...")
        dense_candidates = self.search_dense(query, k=candidate_k)

        if not dense_candidates:
            return []

        # Step 2: Create BM25 scoring on candidates
        candidate_doc_ids = [doc_id for doc_id, _, _ in dense_candidates]
        candidate_indices = [self.doc_id_to_index[doc_id] for doc_id in candidate_doc_ids]

        # Get BM25 scores for query
        tokenized_query = self._preprocess_text(query)
        if not tokenized_query:
            return dense_candidates[:k]

        # Compute BM25 scores for candidates only
        candidate_bm25_scores = {}
        for idx in candidate_indices:
            score = 0
            doc_len = self.bm25.doc_lens[idx]

            for term in tokenized_query:
                if term in self.bm25.doc_term_freqs[idx] and term in self.bm25.idf:
                    tf = self.bm25.doc_term_freqs[idx][term]
                    idf = self.bm25.idf[term]

                    numerator = tf * (self.bm25.k1 + 1)
                    denominator = tf + self.bm25.k1 * (1 - self.bm25.b + self.bm25.b * doc_len / self.bm25.avg_doc_len)
                    score += idf * (numerator / denominator)

            candidate_bm25_scores[self.doc_ids[idx]] = score

        # Step 3: Combine scores
        dense_scores = [score for _, score, _ in dense_candidates]
        bm25_scores = [candidate_bm25_scores.get(doc_id, 0.0) for doc_id, _, _ in dense_candidates]

        # Min-max normalization
        def normalize_scores(scores):
            if not scores or max(scores) == min(scores):
                return [0.0] * len(scores)
            min_s, max_s = min(scores), max(scores)
            return [(s - min_s) / (max_s - min_s) for s in scores]

        norm_dense = normalize_scores(dense_scores)
        norm_bm25 = normalize_scores(bm25_scores)

        # Combine scores
        combined_results = []
        for i, (doc_id, _, text) in enumerate(dense_candidates):
           combined_score = alpha * norm_dense[i] + (1 - alpha) * norm_bm25[i]
           combined_results.append((doc_id, combined_score, text))

       # Sort and return top k
        combined_results.sort(key=lambda x: x[1], reverse=True)
        return combined_results[:k]

    def evaluate_on_dev_set(self, max_queries: int = None, sample_queries: bool = True):
       """Evaluate on MS MARCO dev set with comprehensive metrics"""
       logger.info("Starting evaluation on MS MARCO dev set...")

       if not self.dev_queries or not self.qrels:
           logger.error("No dev queries or qrels loaded")
           return {}

       # Filter queries that have relevance judgments
       valid_queries = {
           qid: qtext for qid, qtext in self.dev_queries.items()
           if qid in self.qrels and any(rel > 0 for rel in self.qrels[qid].values())
       }

       logger.info(f"Found {len(valid_queries)} valid queries with positive relevance judgments")

       # Sample queries for evaluation if needed
       if max_queries and len(valid_queries) > max_queries:
           if sample_queries:
               import random
               random.seed(42)  # For reproducibility
               sample_queries_dict = dict(random.sample(list(valid_queries.items()), max_queries))
               logger.info(f"Randomly sampling {max_queries} queries for evaluation")
           else:
               sample_queries_dict = dict(list(valid_queries.items())[:max_queries])
               logger.info(f"Using first {max_queries} queries for evaluation")
       else:
           sample_queries_dict = valid_queries

       # Evaluate each method
       methods = ['dense', 'bm25', 'hybrid']
       cutoffs = [1, 3, 5, 10, 20, 50, 100]

       results = {}
       for method in methods:
           logger.info(f"Evaluating {method} retrieval on {len(sample_queries_dict)} queries...")
           start_time = time.time()

           method_results = {
               'precision': {k: [] for k in cutoffs},
               'recall': {k: [] for k in cutoffs},
               'mrr': {k: [] for k in cutoffs},
               'ndcg': {k: [] for k in cutoffs},
               'map': [],
               'total_queries': 0,
               'time': 0.0,
               'queries_per_second': 0.0,
               'avg_query_time': 0.0
           }

           query_times = []
           processed_queries = 0

           for query_id, query_text in tqdm(sample_queries_dict.items(), desc=f"Evaluating {method}"):
               try:
                   query_start = time.time()

                   # Get search results
                   if method == 'dense':
                       search_results = self.search_dense(query_text, k=100)
                   elif method == 'bm25':
                       search_results = self.search_bm25(query_text, k=100)
                   elif method == 'hybrid':
                       search_results = self.search_hybrid(query_text, k=100, candidate_k=500, alpha=0.5)
                   else:
                       continue

                   query_times.append(time.time() - query_start)

                   if not search_results:
                       continue

                   retrieved_docs = [doc_id for doc_id, _, _ in search_results]
                   relevant_docs = {doc_id: rel for doc_id, rel in self.qrels[query_id].items() if rel > 0}

                   if not relevant_docs:
                       continue

                   method_results['total_queries'] += 1
                   processed_queries += 1

                   # Calculate metrics for each cutoff
                   for k in cutoffs:
                       retrieved_at_k = retrieved_docs[:k]
                       relevant_retrieved = sum(1 for doc_id in retrieved_at_k if doc_id in relevant_docs)

                       # Precision@k
                       precision = relevant_retrieved / k if k > 0 else 0.0
                       method_results['precision'][k].append(precision)

                       # Recall@k
                       recall = relevant_retrieved / len(relevant_docs) if relevant_docs else 0.0
                       method_results['recall'][k].append(recall)

                       # MRR@k
                       mrr = 0.0
                       for i, doc_id in enumerate(retrieved_at_k):
                           if doc_id in relevant_docs:
                               mrr = 1.0 / (i + 1)
                               break
                       method_results['mrr'][k].append(mrr)

                       # NDCG@k
                       dcg = sum(
                           (2**min(relevant_docs.get(doc_id, 0), 3) - 1) / np.log2(i + 2)
                           for i, doc_id in enumerate(retrieved_at_k)
                           if doc_id in relevant_docs
                       )
                       ideal_relevances = sorted(relevant_docs.values(), reverse=True)[:k]
                       idcg = sum((2**min(rel, 3) - 1) / np.log2(i + 2) for i, rel in enumerate(ideal_relevances))
                       ndcg = dcg / idcg if idcg > 0 else 0.0
                       method_results['ndcg'][k].append(ndcg)

                   # MAP calculation
                   if relevant_docs:
                       ap = 0.0
                       rel_found = 0
                       for i, doc_id in enumerate(retrieved_docs):
                           if doc_id in relevant_docs:
                               rel_found += 1
                               ap += rel_found / (i + 1)
                       map_score = ap / len(relevant_docs)
                       method_results['map'].append(map_score)

                   # Log progress every 100 queries
                   if processed_queries % 100 == 0:
                       logger.info(f"Processed {processed_queries} queries for {method}...")

               except Exception as e:
                   logger.warning(f"Error processing query {query_id}: {e}")
                   continue

           # Calculate final averages
           for metric_name in ['precision', 'recall', 'mrr', 'ndcg']:
               for k in cutoffs:
                   values = method_results[metric_name][k]
                   method_results[metric_name][k] = np.mean(values) if values else 0.0

           map_values = method_results['map']
           method_results['map'] = np.mean(map_values) if map_values else 0.0
           method_results['time'] = time.time() - start_time

           if query_times:
               method_results['queries_per_second'] = len(query_times) / sum(query_times)
               method_results['avg_query_time'] = np.mean(query_times)

           results[method] = method_results

           logger.info(f"{method} evaluation completed: {method_results['total_queries']} queries, "
                       f"{method_results['queries_per_second']:.2f} queries/sec, "
                       f"MAP: {method_results['map']:.4f}")

       return results

    def print_evaluation_results(self, results: Dict[str, Any]):
       """Print comprehensive evaluation results"""
       print("\n" + "="*120)
       print("MS MARCO PASSAGE RETRIEVAL EVALUATION RESULTS")
       print("="*120)

       methods = ['dense', 'bm25', 'hybrid']
       cutoffs = [1, 3, 5, 10, 20, 50, 100]

       for method in methods:
           if method not in results:
               continue

           metrics = results[method]
           print(f"\n{method.upper()} RETRIEVAL RESULTS:")
           print("-" * 60)
           print(f"Total Queries Evaluated: {metrics.get('total_queries', 0)}")
           print(f"Mean Average Precision (MAP): {metrics['map']:.4f}")
           print(f"Evaluation Time: {metrics.get('time', 0):.1f} seconds")
           print(f"Queries per Second: {metrics.get('queries_per_second', 0):.2f}")
           print(f"Average Query Time: {metrics.get('avg_query_time', 0):.4f} seconds\n")

           print(f"{'Metric':<12} {'@1':<8} {'@3':<8} {'@5':<8} {'@10':<8} {'@20':<8} {'@50':<8} {'@100':<8}")
           print("-" * 88)

           for metric_name in ['precision', 'recall', 'mrr', 'ndcg']:
               values = [f"{metrics[metric_name][k]:.4f}" for k in cutoffs]
               print(f"{metric_name.upper():<12} {' '.join(f'{v:<8}' for v in values)}")

       # Summary comparison
       print("\n" + "="*120)
       print("METHOD COMPARISON SUMMARY")
       print("="*120)
       print(f"{'Method':<15} {'MAP':<10} {'MRR@10':<10} {'Recall@10':<12} {'NDCG@10':<10} {'QPS':<10} {'Time(s)':<10}")
       print("-" * 80)

       for method in methods:
           if method in results:
               metrics = results[method]
               print(f"{method.upper():<15} {metrics['map']:<10.4f} "
                     f"{metrics['mrr'][10]:<10.4f} "
                     f"{metrics['recall'][10]:<12.4f} "
                     f"{metrics['ndcg'][10]:<10.4f} "
                     f"{metrics.get('queries_per_second', 0):<10.2f} "
                     f"{metrics.get('time', 0):<10.1f}")

       print("\n" + "="*120)

       # Additional insights
       if 'hybrid' in results and 'dense' in results and 'bm25' in results:
           hybrid_map = results['hybrid'].get('map', 0.0)
           dense_map = results['dense'].get('map', 0.0)
           bm25_map = results['bm25'].get('map', 0.0)

           print("PERFORMANCE INSIGHTS:")
           if dense_map > 0:
               improvement = ((hybrid_map - dense_map) / dense_map * 100)
               print(f"Hybrid vs Dense MAP improvement: {improvement:+.1f}%")
           if bm25_map > 0:
               improvement = ((hybrid_map - bm25_map) / bm25_map * 100)
               print(f"Hybrid vs BM25 MAP improvement: {improvement:+.1f}%")

           # Find best method
           best_method = max(methods, key=lambda m: results[m].get('map', -1) if m in results else -1)
           if best_method in results:
               print(f"Best performing method: {best_method.upper()} (MAP: {results[best_method]['map']:.4f})")

           print("\nHybrid Search Configuration:")
           print(f"- Dense candidates: 500")
           print(f"- BM25 re-ranking weight: 0.5")
           print(f"- Final results returned: 100")

       print("="*120)

def main():
   """Main function for full MS MARCO evaluation"""
   logger.info("="*80)
   logger.info("STARTING FULL MS MARCO EVALUATION SYSTEM")
   logger.info("This will download and process the COMPLETE MS MARCO dataset")
   logger.info("="*80)

   try:
       # Create evaluator
       evaluator = MSMarcoEvaluator(
           model_name="all-MiniLM-L6-v2",
           save_dir="./msmarco_full_eval"
       )

       # Step 1: Load the complete dataset
       logger.info("STEP 1: Loading complete MS MARCO dataset...")
       success = evaluator.load_full_msmarco_data()
       if not success:
           logger.error("Failed to load MS MARCO dataset")
           return

       logger.info("Dataset loaded successfully!")
       logger.info(f"- Documents: {len(evaluator.documents):,}")
       logger.info(f"- Queries: {len(evaluator.dev_queries):,}")
       logger.info(f"- QRels: {len(evaluator.qrels):,}")

       # Step 2: Build dense retrieval index with IVF
       logger.info("STEP 2: Building dense retrieval index with IVF centroids...")
       evaluator.build_dense_index(batch_size=64)
       logger.info("Dense index with IVF built successfully!")

       # Step 3: Build BM25 index
       logger.info("STEP 3: Building BM25 sparse retrieval index...")
       evaluator.build_bm25_index()
       logger.info("BM25 index built successfully!")

       # Step 4: Test search methods
       logger.info("STEP 4: Testing search methods...")
       test_query = "what is machine learning algorithm"

       # Test dense search
       logger.info("Testing dense search...")
       dense_results = evaluator.search_dense(test_query, k=10)
       logger.info(f"Dense search: Retrieved {len(dense_results)} results")
       if dense_results:
           logger.info(f"Top result score: {dense_results[0][1]:.4f}")

       # Test BM25 search
       logger.info("Testing BM25 search...")
       bm25_results = evaluator.search_bm25(test_query, k=10)
       logger.info(f"BM25 search: Retrieved {len(bm25_results)} results")
       if bm25_results:
           logger.info(f"Top result score: {bm25_results[0][1]:.4f}")

       # Test hybrid search (500 candidates from dense + BM25 reranking)
       logger.info("Testing hybrid search (500 dense candidates + BM25 reranking)...")
       hybrid_results = evaluator.search_hybrid(test_query, k=10, candidate_k=500, alpha=0.5)
       logger.info(f"Hybrid search: Retrieved {len(hybrid_results)} results")
       if hybrid_results:
           logger.info(f"Top result score: {hybrid_results[0][1]:.4f}")

       # Step 5: Evaluation on dev set
       logger.info("STEP 5: Starting evaluation on MS MARCO dev set...")
       
       # Start with subset for quick test
       logger.info("Running evaluation on 1000 queries for initial testing...")
       results = evaluator.evaluate_on_dev_set(max_queries=1000, sample_queries=True)

       # Print results
       evaluator.print_evaluation_results(results)

       # Save initial results
       results_path = os.path.join(evaluator.save_dir, 'evaluation_results_1000.json')
       with open(results_path, 'w') as f:
           json.dump(results, f, indent=2)
       logger.info(f"Results saved to {results_path}")

       # Option for full evaluation
       print("\nDo you want to run full evaluation on ALL dev queries? (y/n): ", end="")
       try:
           response = input().lower().strip()
           if response == 'y':
               logger.info("Starting FULL evaluation on all dev queries...")
               logger.info("This may take several hours depending on your hardware...")
               
               full_results = evaluator.evaluate_on_dev_set(max_queries=None)
               evaluator.print_evaluation_results(full_results)
               
               # Save full results
               full_results_path = os.path.join(evaluator.save_dir, 'full_evaluation_results.json')
               with open(full_results_path, 'w') as f:
                   json.dump(full_results, f, indent=2)
               logger.info(f"Full results saved to {full_results_path}")
               
       except KeyboardInterrupt:
           logger.info("Evaluation interrupted by user")

       logger.info("="*80)
       logger.info("EVALUATION COMPLETED SUCCESSFULLY!")
       logger.info("All data has been cached for future runs")
       logger.info("="*80)

   except KeyboardInterrupt:
       logger.info("Evaluation interrupted by user")
   except Exception as e:
       logger.error(f"Critical error in main execution: {e}", exc_info=True)
       raise

if __name__ == "__main__":
   main()

2025-07-20 04:04:05,073 - INFO - ================================================================================
2025-07-20 04:04:05,074 - INFO - STARTING FULL MS MARCO EVALUATION SYSTEM
2025-07-20 04:04:05,075 - INFO - This will download and process the COMPLETE MS MARCO dataset
2025-07-20 04:04:05,075 - INFO - ================================================================================
2025-07-20 04:04:05,077 - INFO - Initializing MS MARCO Evaluator with save directory: ./msmarco_full_eval
2025-07-20 04:04:05,078 - INFO - Loading sentence transformer model: all-MiniLM-L6-v2
2025-07-20 04:04:05,086 - INFO - Use pytorch device_name: cpu
2025-07-20 04:04:05,087 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-07-20 04:04:08,744 - INFO - Model loaded successfully. Max sequence length: 256
2025-07-20 04:04:08,745 - INFO - Using CPU for inference
2025-07-20 04:04:08,745 - INFO - STEP 1: Loading complete MS MARCO dataset...
2025-07-20 04:04:08,745 - INFO - Loading FU